In [1]:
# Try to use the solution at:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3079956/

In [2]:
import numpy as np
from scipy.special import ellipe, ellipk, ellipeinc
%matplotlib inline
import matplotlib.pylab as plt

In [3]:
# try to iteratively solve for m

def k_m(m, n):
    """
    Parameters
    ----------
    m : float
        The value to iterate
    n : int
        The number of points on the sphere
    """
    numerator = m*np.pi*n
    numerator *= (2.*ellipe(-m**2) - ellipk(-m**2))
    denom = n*np.pi*ellipe(-m**2)
    denom -= n*np.pi*ellipk(-m**2)
    denom += m*ellipe(-m**2)**2
    result = numerator/denom
    return result

In [4]:
# So, I should wrap all this up with a tolerance kwarg
n = 20.
# Let's try to iterate this thing
m_old = np.sqrt(n*np.pi)
ms = [m_old]
for i in range(10):
    ms.append(k_m(ms[-1], n))
    

In [5]:
def S_length(theta, m):
    """
    Where theta is an array
    """
    case1 = np.where((theta > np.pi/2.) & (theta <= np.pi))
    case2 = np.where((theta >= 0.) & (theta <= np.pi/2.))
    
    result = theta*0
    result[case1] = 2.*ellipe(-m**2) - ellipeinc(np.pi-theta[case1] , -m**2)
    result[case2] = ellipeinc(theta[case2], -m**2)
    return result

In [6]:
def k_theta(theta, m):
    """
    Parameters
    """
    # I could make this a class and make j just once
    j = np.arange(theta.size)+1
    result = theta+0
    numerator = (2.*j-1)*np.pi - m*S_length(theta, m)
    denom = m*np.sqrt(1.+m**2*np.sin(theta)**2)
    result += numerator/denom
    # Make sure we don't wrap around in crazy ways
    result = result % (np.pi)
    return result


In [7]:
j = np.arange(n)+1
theta_init = np.arccos(1.-(2*j-1)/n)

In [8]:
theta = theta_init
for i in range(10):
    theta = k_theta(theta, ms[-1])

In [9]:
phi = ms[-1]*theta % (2.*np.pi)  # phi is longitude (or RA). z=cos(theta). so dec = pi/2 - theta

In [10]:
def theta2dec(theta):
    dec = np.pi/2. - theta
    return dec

In [11]:
theta - k_theta(theta, ms[-1])

array([  0.00000000e+00,   0.00000000e+00,  -1.11022302e-16,
         1.11022302e-16,   0.00000000e+00,   2.22044605e-16,
         2.22044605e-16,  -4.44089210e-16,   2.22044605e-16,
         2.22044605e-16,  -2.22044605e-16,   2.22044605e-16,
         0.00000000e+00,   0.00000000e+00,   4.44089210e-16,
         4.44089210e-16,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   8.88178420e-16])

In [12]:
dec = theta2dec(theta)

In [13]:
dec

array([ 1.30347023,  1.04858044,  0.87258193,  0.726781  ,  0.59749671,
        0.47853012,  0.36639067,  0.25882034,  0.15419281,  0.05121951,
       -0.05121951, -0.15419281, -0.25882034, -0.36639067, -0.47853012,
       -0.59749671, -0.726781  , -0.87258193, -1.04858044, -1.30347023])

In [14]:
phi

array([ 2.08555975,  4.074097  ,  5.44715941,  0.30144839,  1.31006713,
        2.23819182,  3.11305407,  3.95227013,  4.76852782,  5.57187988,
        0.08787839,  0.89123046,  1.70748815,  2.54670421,  3.42156646,
        4.34969115,  5.35830989,  0.21259887,  1.58566128,  3.57419853])

In [ ]:
# let's wrap it up in a class
class make_spiral(object, ttol=1e-10):
    def __init__(self, n):
        self.n = n
        m_init = np.sqrt(n*np.pi)
        
        # iterate to solve for m
        
        # 
        
        
    def k_m(m):
        """
        Appendix A
        
        Parameters
        ----------
        m : float
            The value to iterate
        """
        numerator = m*np.pi*self.n
        numerator *= (2.*ellipe(-m**2) - ellipk(-m**2))
        denom = self.n*np.pi*ellipe(-m**2)
        denom -= self.n*np.pi*ellipk(-m**2)
        denom += m*ellipe(-m**2)**2
        result = numerator/denom
        return result
    